In [4]:
import pandas as pd
from matplotlib import pyplot as plt
from tabulate import tabulate
from pathlib import Path
from typing import Sequence, Tuple, Dict, Any
import numpy as np
import re
import locale
locale.setlocale(locale.LC_ALL, '')
from scipy.optimize import minimize
from matplotlib import rcParams
rcParams['figure.figsize'] = 9, 9
import sys
import subprocess

from phz_helpers import make_valid_filename#, extract_rms, create_phz, 
from export_wichert_to_phazecomp7 import export_wichert_to_phazecomp

from norm_denorm import norm_x0, denorm_solx
#Phz_loc = r"C:\Temp\PhazeComp\phazecomp.exe"
#Phz_loc = r"C:\Program Files\PhazeComp\1.81\phazecomp.exe"
#Phz_loc = r"C:\Program Files\PhazeComp\PhazeComp_1.95.exe"
Phz_loc = 'C:\\Users\\vinom\\OneDrive - Santos\\PVT\\PhazeComp\\PhazeComp\\PhazeComp Application\\PhazeComp 1.9.5\\phazecomp.exe'
R, mwAir, degF2R = 10.731577089016, 28.97, 459.67
experiment_dir = "combined_experiments_10"

In [5]:
def create_phz(phz_template):
    rms=float("nan")
    try:
        proc = subprocess.run([Phz_loc, phz_template+'.Phz','NUL',">"], capture_output=True)
    except:
        print("Either phazecomp.exe couldnt be found (is it in "+Phz_loc+" ?), or "+phz_template+" couldn't be found - Exiting..")
        sys.exit()
    
    found_rms = False
    for i in range(len(proc.stderr.splitlines())-1,0,-1):
        line = proc.stderr.splitlines()[i].decode("utf-8")
        if "% RMS average weighted" in line:
            rms = float(line.split()[2])
            found_rms = True
            break
    if not found_rms:
        print("Error: No RMS for ",phz_template)
    return rms

## Export Wichert raw experiment data. One experiment per temperature

In [6]:
sample_dic = export_wichert_to_phazecomp(xls_path="tabulated_data6.xlsm", output_subfolder=experiment_dir)

INCLUDE "combined_experiments_10\McLeod-Mix1.inc"
INCLUDE "combined_experiments_10\McLeod-Mix2.inc"
INCLUDE "combined_experiments_10\McLeod-Mix3.inc"
INCLUDE "combined_experiments_10\McLeod-Mix22.inc"
INCLUDE "combined_experiments_10\McLeod-Mix24.inc"
INCLUDE "combined_experiments_10\McLeod-Mix25.inc"
INCLUDE "combined_experiments_10\Simon&Briggs-10PC.inc"
INCLUDE "combined_experiments_10\Simon&Briggs-20PC.inc"
INCLUDE "combined_experiments_10\API 37-Mix2.inc"
INCLUDE "combined_experiments_10\API 37-Mix17.inc"
INCLUDE "combined_experiments_10\API 37-Mix25.inc"
INCLUDE "combined_experiments_10\API 37-Mix26.inc"
INCLUDE "combined_experiments_10\API 37-Mix402.inc"
INCLUDE "combined_experiments_10\API 37-Mix601.inc"
INCLUDE "combined_experiments_10\API 37-Mix705.inc"
INCLUDE "combined_experiments_10\API 37-Mix707.inc"
INCLUDE "combined_experiments_10\API 37-Mix722.inc"
INCLUDE "combined_experiments_10\API 37-Mix727.inc"
INCLUDE "combined_experiments_10\API 37-Mix729.inc"
INCLUDE "combined_

## Define new Tc & Pc relationship based on Sutton original and AG shapes
But fitting to Langmuir form, and using Tc with striaght line Vc/Zc

In [7]:
tc_offset, mw_offset, vc_offset = 343.008, 16.042, 5.518526

def tc_ag(x, tc_params):
    a, b, c = tc_params[1]
    return a * x / (b + x) + c

def tc_gc(x, tc_params):
    a, b, c = tc_params[0]
    return a * x / (b + x) + c

tc_fn = {1: tc_ag,
         0: tc_gc}

def pc_fn(x, vc_on_zc_params, tc, AG):
    slope2, slope1 = vc_on_zc_params[AG]
#    delta, shared_slope, slope1 = vc_on_zc_params[AG]
#    if x < delta:
#        vc_on_zc = vc_offset + shared_slope * x
#    else:
#        # At x=delta: value is T + shared_slope * delta
#        # For x>delta, start from there, continue with slope1
#        vc_on_zc = vc_offset + shared_slope * delta + slope1 * (x - delta)
    
    vc_on_zc = slope2*x**2 + slope1 * x + vc_offset
    return R * tc / vc_on_zc
    
def tc_pc(mw, tc_params, vc_on_zc_params, AG):
    x = max(0, mw - mw_offset)
#    tcp_ag = tc_fn[True](x, tc_params)
#    tcp_gc = tc_fn[False](x, tc_params)
#    if AG:
#        tc_p = max(tcp_ag, tcp_gc)
#    else:
#        tc_p = min(tcp_ag, tcp_gc)
    if AG:
        tc_p = tc_fn[True](x, tc_params)
    else:
        tc_p = tc_fn[False](x, tc_params)
    pc_p = pc_fn(x, vc_on_zc_params, tc_p, AG)
    return tc_p, pc_p

## Run the regression - Wichert Data

In [13]:
# Constants (anchor to methane)
MW_CH4      = 16.043
TC_CH4      = 343.008
PC_CH4      = 667.029
VC_CH4      = 1.579420001
ZC_CH4      = 0.286203243
VC_OVER_ZC  = VC_CH4 / ZC_CH4
R_CONST     = 10.7316      # (psia·ft³)/(lb·mol·°R)
DEGF2R      = 459.67       # °F → °R offset
COMPS       = ['CO2', 'H2S', 'N2', 'H2', 'Gas']
BASE_CHR    = 'RealGas'
GERG_MWS    = [16.043, 27.91, 34.51, 41.63]
GERG_NAMES  = ['lean_gas','leanmid_gas','mid_gas','rich_gas']
GERG_TEMPS  = [60, 90, 120, 150, 180, 220, 250, 280]
mws   = np.array([44.01, 34.082, 28.014, 2.016, 0.])
MW_Air = 28.97

def write_eos_includes(
    x: np.ndarray,
    sample_dic: Dict[str, Any],
    output_dir: str,
    phz_template: str
) -> float:
    """
    1) Generates new EOS .inc files for each sample/temp
    2) Runs PhazeComp (via create_phz) and returns the final RMS
    """
    x = denorm_solx(x, bounds)
    slope2_ag, slope2_gc, slope1_ag, slope1_gc, a_ag, b_ag, a_gc, b_gc = x
    slope2_ag, slope2_gc
    c_ag = tc_offset
    c_gc = tc_offset
    tc_params = {1: [a_ag, b_ag, c_ag], # Associated Gas
                 0: [a_gc, b_gc, c_gc]} # Gas condensate
    
    vc_on_zc_params = {1: [slope2_ag, slope1_ag], # Associated Gas
                       0: [slope2_gc, slope1_gc]} # Gas condensate
    
    output_dir = Path(output_dir)

    # Loop over each sample
    for name in list(sample_dic):
        meta = sample_dic[name]
        sg, y_co2, y_h2s, y_n2, y_h2, y_gas = meta['sg'], meta['co2'], meta['h2s'], meta['n2'], 0, meta['gas']
        c7_on_c2 = meta['c7_on_c2']
        degfs = meta['degfs']
        exps = meta['exps']
        zi    = np.array([y_co2, y_h2s, y_n2, y_h2, y_gas])

        # compute pure hydrocarbon gas MW
        mw_hc = (sg - (zi[:4]*mws[:4]).sum()/MW_Air)/(1-zi[:4].sum()) * MW_Air
        mws[-1] = mw_hc

        # get Tc,Pc for the Gas component
        if c7_on_c2 > 0.65:
            AG_fraction = 1 #ramp_between_limits(mw_hc, min_val, max_val)
        else:
            AG_fraction = 0

        tc_gas, pc_gas = tc_pc(mw_hc, tc_params, vc_on_zc_params, AG_fraction)

        # --- build & write sample include ---
        #lines = [f'; AG Params: ag_slope2 {ag_slope2:.6f}, ag_slope {ag_slope:.6f}, ag_vl {ag_vl:.6f}, ag_pl {ag_pl:.6f}']
        #lines.append(f'; DG Params: dg_slope2 {dg_slope2:.6f}, dg_slope {dg_slope:.6f}, dg_vl {dg_vl:.6f}, dg_pl {dg_pl:.6f}\n')
        lines = []
        lines.append(f'DEFINE MW {mw_hc:.6f} , DEFINE Gas_Tc {tc_gas:.6f} ; AG Fraction {AG_fraction:.6f}, C7+/C2 = {c7_on_c2:.3f}')
        lines.append(f'CHARACTERIZATION "CHR_{make_valid_filename(name)}" BASE "{BASE_CHR}"')

        # COMPONENT table
        comp_rows = [['COMPONENT','MW','TC,R','PC,PSIA','AF','VTRAN','A','B']]
        for comp, mw_val in zip(COMPS, mws):
            row = [comp, '', '', '', '', '', '', '']
            if comp == 'Gas':
                row[1] = f'{mw_hc:.4f}'
                row[2] = f'{tc_gas:.4f}'
                row[3] = f'{pc_gas:.4f}'
            comp_rows.append(row)
        lines.append(tabulate(comp_rows, tablefmt='plain'))
        lines.append('END\n')

        # MIX block
        mix_rows = [['MIX',make_valid_filename(name)]] + [[c, f'{zi[i]:.6f}'] for i,c in enumerate(COMPS)]
        lines.append(tabulate(mix_rows, tablefmt='plain'))
        lines.append('END\n')

        # Temperature‐dependent CHARACTERIZATIONS
        for t, temp in enumerate(degfs):
            degR = temp + DEGF2R
            lines.append(f'DEFINE degR {degR:.6f}')
            lines.append(f'CHARACTERIZATION "CHR_{make_valid_filename(name)}_{temp}" BASE "CHR_{make_valid_filename(name)}"')
            lines.append(tabulate(comp_rows, tablefmt='plain'))
            lines.append('END\n')
            lines.append('INCLUDE bips_calc3.inc\n')

            # feed + include exp
            lines.append(f'QIX FEED 1 MOLE {make_valid_filename(name)}')
            lines.append('INCLUDE "'+name+'-'+str(int(round(temp,0)))+'.inc"\n')
        
        output_dir.mkdir(parents=True, exist_ok=True)
        file_path = output_dir / (name+'.inc')
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(('\n').join(lines))

        #print('INCLUDE "'+name+'.inc"')
    
    # Create MIX_VARS.INC
    mix_mws = [37.105, 40.394, 30.16]
    mix_names = ['63169', '58647', '4932']
    var_out = ''
    for i, mw_hc in enumerate(mix_mws):  
        # get Tc,Pc for the Gas component
        AG_fraction = 0 # Duvernay shale is gas condensate
        tc_gas, pc_gas = tc_pc(mw_hc, tc_params, vc_on_zc_params, AG_fraction)

        var_out += 'VARIABLE Tc_'+mix_names[i]+' '+str(tc_gas)+' ; AG_Fraction '+str(AG_fraction)+'\n'
        var_out += 'VARIABLE Pc_'+mix_names[i]+' '+str(pc_gas)+'\n'
    file_path = output_dir / ('MIX_VARS.INC')
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(var_out)
        
    
    # finally run PhazeComp & grab RMS
    rms = create_phz(phz_template)
    print(f'{x} → RMS {rms:.3f}%')
    return rms

In [14]:
experiment_dir = "combined_experiments_9"
deck_name = 'Tr_BIPs_paramterised_9_Combined'

x0 = [0, 0, 1.78007611e-01, 1.64110319e-01, 2.64825909e+03, 2.65850145e+02, 1.02121422e+03, 9.96109315e+01]
bounds = [(-1e-4, 1e-4), (-1e-4, 1e-4), (0.15, 0.2), (0.15, 0.2), (1000, 5000), (100, 500), (700, 2000), (50, 200)]
x = norm_x0(x0, bounds)

# Suppose x0 is your initial 1-D ndarray of length n
n = len(x)

# Build a simplex that’s 10% off in every direction
spread = 0.1
simp = np.zeros((n+1, n))
simp[0] = x
for i in range(n):
    v = x.copy()
    v[i] += spread * max(1.0, abs(x[i]))
    simp[i+1] = v

res = minimize(
    write_eos_includes, x,
    method="Nelder-Mead",
    args = (sample_dic, experiment_dir, deck_name),
    bounds = [(0, 1) for _ in range(n)],
    options={
      "initial_simplex": simp,
      "xatol": 1e-4,     # keep these small enough to finish
      "fatol": 1e-4,
      "maxiter": 2000,
    }
)

[0.00000000e+00 0.00000000e+00 1.78007611e-01 1.64110319e-01
 2.64825909e+03 2.65850145e+02 1.02121422e+03 9.96109315e+01] → RMS 1.233%
[2.00000000e-05 0.00000000e+00 1.78007611e-01 1.64110319e-01
 2.64825909e+03 2.65850145e+02 1.02121422e+03 9.96109315e+01] → RMS 1.239%
[0.00000000e+00 2.00000000e-05 1.78007611e-01 1.64110319e-01
 2.64825909e+03 2.65850145e+02 1.02121422e+03 9.96109315e+01] → RMS 1.233%
[0.00000000e+00 0.00000000e+00 1.83007611e-01 1.64110319e-01
 2.64825909e+03 2.65850145e+02 1.02121422e+03 9.96109315e+01] → RMS 1.406%
[0.00000000e+00 0.00000000e+00 1.78007611e-01 1.69110319e-01
 2.64825909e+03 2.65850145e+02 1.02121422e+03 9.96109315e+01] → RMS 1.243%
[0.00000000e+00 0.00000000e+00 1.78007611e-01 1.64110319e-01
 3.04825909e+03 2.65850145e+02 1.02121422e+03 9.96109315e+01] → RMS 2.008%
[0.00000000e+00 0.00000000e+00 1.78007611e-01 1.64110319e-01
 2.64825909e+03 3.05850145e+02 1.02121422e+03 9.96109315e+01] → RMS 1.823%
[0.00000000e+00 0.00000000e+00 1.78007611e-01 1.

[-3.32270721e-06  7.98687069e-06  1.77995610e-01  1.65439608e-01
  2.66078345e+03  2.67018851e+02  1.03217690e+03  9.89979376e+01] → RMS 1.235%
[2.91091112e-06 5.73830559e-06 1.78136034e-01 1.64957811e-01
 2.65583199e+03 2.66471551e+02 1.02520671e+03 9.99148616e+01] → RMS 1.234%
[-7.03435138e-07  1.13129914e-05  1.78170906e-01  1.62829690e-01
  2.66001489e+03  2.66979816e+02  1.02881164e+03  9.97799300e+01] → RMS 1.235%
[1.68248498e-06 4.68659161e-06 1.78067659e-01 1.65895373e-01
 2.65634856e+03 2.66512578e+02 1.02673306e+03 9.95953180e+01] → RMS 1.234%
[9.18715676e-08 9.10419145e-06 1.78136490e-01 1.63851584e-01
 2.65879278e+03 2.66824070e+02 1.02811878e+03 9.97183927e+01] → RMS 1.234%
[1.28483163e-06 5.79099157e-06 1.78084867e-01 1.65384426e-01
 2.65695961e+03 2.66590451e+02 1.02707949e+03 9.96260867e+01] → RMS 1.234%
[0.00000000e+00 1.00000000e-05 1.78007611e-01 1.64110319e-01
 2.64825909e+03 2.65850145e+02 1.02121422e+03 9.96109315e+01] → RMS 1.233%
[8.74184325e-07 2.86648761e-06 1

[0.00000000e+00 3.12500000e-07 1.78007611e-01 1.64110319e-01
 2.64825909e+03 2.65850145e+02 1.02121422e+03 9.96109315e+01] → RMS 1.233%
[2.73182602e-08 8.95777377e-08 1.78008369e-01 1.64125646e-01
 2.64842210e+03 2.65864375e+02 1.02137837e+03 9.96118069e+01] → RMS 1.233%
[-8.89428371e-08  1.28647495e-07  1.78009785e-01  1.64131814e-01
  2.64849856e+03  2.65870739e+02  1.02138385e+03  9.96117609e+01] → RMS 1.233%
[3.85830286e-08 7.90850553e-08 1.78008192e-01 1.64125222e-01
 2.64845191e+03 2.65879873e+02 1.02137664e+03 9.96141698e+01] → RMS 1.233%
[4.54829863e-08 8.96610249e-08 1.78009618e-01 1.64123561e-01
 2.64837742e+03 2.65859854e+02 1.02127660e+03 9.96156804e+01] → RMS 1.233%
[2.62888278e-08 7.32279939e-08 1.78008549e-01 1.64138210e-01
 2.64838549e+03 2.65860496e+02 1.02130045e+03 9.96106875e+01] → RMS 1.233%
[2.83095243e-08 9.48596934e-08 1.78010452e-01 1.64125123e-01
 2.64840126e+03 2.65862175e+02 1.02130079e+03 9.96090178e+01] → RMS 1.233%
[-7.22822837e-09  1.23055510e-07  1.7800

## And linear Vc/Zc - This is the solution

In [8]:
# Constants (anchor to methane)
MW_CH4      = 16.043
TC_CH4      = 343.008
PC_CH4      = 667.029
VC_CH4      = 1.579420001
ZC_CH4      = 0.286203243
VC_OVER_ZC  = VC_CH4 / ZC_CH4
R_CONST     = 10.7316      # (psia·ft³)/(lb·mol·°R)
DEGF2R      = 459.67       # °F → °R offset
COMPS       = ['CO2', 'H2S', 'N2', 'H2', 'Gas']
BASE_CHR    = 'RealGas'
GERG_MWS    = [16.043, 27.91, 34.51, 41.63]
GERG_NAMES  = ['lean_gas','leanmid_gas','mid_gas','rich_gas']
GERG_TEMPS  = [60, 90, 120, 150, 180, 220, 250, 280]
mws   = np.array([44.01, 34.082, 28.014, 2.016, 0.])
MW_Air = 28.97

def write_eos_includes(
    x: np.ndarray,
    sample_dic: Dict[str, Any],
    output_dir: str,
    phz_template: str
) -> float:
    """
    1) Generates new EOS .inc files for each sample/temp
    2) Runs PhazeComp (via create_phz) and returns the final RMS
    """
    x = denorm_solx(x, bounds)
    slope1_ag, slope1_gc, a_ag, b_ag, a_gc, b_gc = x
    slope2_ag, slope2_gc = 0, 0
    c_ag = tc_offset
    c_gc = tc_offset
    tc_params = {1: [a_ag, b_ag, c_ag], # Associated Gas
                 0: [a_gc, b_gc, c_gc]} # Gas condensate
    
    vc_on_zc_params = {1: [slope2_ag, slope1_ag], # Associated Gas
                       0: [slope2_gc, slope1_gc]} # Gas condensate
    
    output_dir = Path(output_dir)

    # Loop over each sample
    for name in list(sample_dic):
        meta = sample_dic[name]
        sg, y_co2, y_h2s, y_n2, y_h2, y_gas = meta['sg'], meta['co2'], meta['h2s'], meta['n2'], 0, meta['gas']
        c7_on_c2 = meta['c7_on_c2']
        degfs = meta['degfs']
        exps = meta['exps']
        zi    = np.array([y_co2, y_h2s, y_n2, y_h2, y_gas])

        # compute pure hydrocarbon gas MW
        mw_hc = (sg - (zi[:4]*mws[:4]).sum()/MW_Air)/(1-zi[:4].sum()) * MW_Air
        mws[-1] = mw_hc

        # get Tc,Pc for the Gas component
        if c7_on_c2 > 0.65:
            AG_fraction = 1 #ramp_between_limits(mw_hc, min_val, max_val)
        else:
            AG_fraction = 0

        tc_gas, pc_gas = tc_pc(mw_hc, tc_params, vc_on_zc_params, AG_fraction)

        # --- build & write sample include ---
        #lines = [f'; AG Params: ag_slope2 {ag_slope2:.6f}, ag_slope {ag_slope:.6f}, ag_vl {ag_vl:.6f}, ag_pl {ag_pl:.6f}']
        #lines.append(f'; DG Params: dg_slope2 {dg_slope2:.6f}, dg_slope {dg_slope:.6f}, dg_vl {dg_vl:.6f}, dg_pl {dg_pl:.6f}\n')
        lines = []
        lines.append(f'DEFINE MW {mw_hc:.6f} , DEFINE Gas_Tc {tc_gas:.6f} ; AG Fraction {AG_fraction:.6f}, C7+/C2 = {c7_on_c2:.3f}')
        lines.append(f'CHARACTERIZATION "CHR_{make_valid_filename(name)}" BASE "{BASE_CHR}"')

        # COMPONENT table
        comp_rows = [['COMPONENT','MW','TC,R','PC,PSIA','AF','VTRAN','A','B']]
        for comp, mw_val in zip(COMPS, mws):
            row = [comp, '', '', '', '', '', '', '']
            if comp == 'Gas':
                row[1] = f'{mw_hc:.4f}'
                row[2] = f'{tc_gas:.4f}'
                row[3] = f'{pc_gas:.4f}'
            comp_rows.append(row)
        lines.append(tabulate(comp_rows, tablefmt='plain'))
        lines.append('END\n')

        # MIX block
        mix_rows = [['MIX',make_valid_filename(name)]] + [[c, f'{zi[i]:.6f}'] for i,c in enumerate(COMPS)]
        lines.append(tabulate(mix_rows, tablefmt='plain'))
        lines.append('END\n')

        # Temperature‐dependent CHARACTERIZATIONS
        for t, temp in enumerate(degfs):
            degR = temp + DEGF2R
            lines.append(f'DEFINE degR {degR:.6f}')
            lines.append(f'CHARACTERIZATION "CHR_{make_valid_filename(name)}_{temp}" BASE "CHR_{make_valid_filename(name)}"')
            lines.append(tabulate(comp_rows, tablefmt='plain'))
            lines.append('END\n')
            lines.append('INCLUDE bips_calc3.inc\n')

            # feed + include exp
            lines.append(f'QIX FEED 1 MOLE {make_valid_filename(name)}')
            lines.append('INCLUDE "'+name+'-'+str(int(round(temp,0)))+'.inc"\n')
        
        output_dir.mkdir(parents=True, exist_ok=True)
        file_path = output_dir / (name+'.inc')
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(('\n').join(lines))

        #print('INCLUDE "'+name+'.inc"')
    
    # Create MIX_VARS.INC
    mix_mws = [37.105, 40.394, 30.16]
    mix_names = ['63169', '58647', '4932']
    var_out = ''
    for i, mw_hc in enumerate(mix_mws):  
        # get Tc,Pc for the Gas component
        AG_fraction = 0 # Duvernay shale is gas condensate
        tc_gas, pc_gas = tc_pc(mw_hc, tc_params, vc_on_zc_params, AG_fraction)

        var_out += 'VARIABLE Tc_'+mix_names[i]+' '+str(tc_gas)+' ; AG_Fraction '+str(AG_fraction)+'\n'
        var_out += 'VARIABLE Pc_'+mix_names[i]+' '+str(pc_gas)+'\n'
    file_path = output_dir / ('MIX_VARS.INC')
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(var_out)
        
    
    # finally run PhazeComp & grab RMS
    rms = create_phz(phz_template)
    print(f'{x} → RMS {rms:.3f}%')
    return rms

In [9]:
experiment_dir = "combined_experiments_10"
deck_name = 'Tr_BIPs_paramterised_10_Combined'

x0 = [1.77497520e-01, 1.70930780e-01, 2.69506532e+03, 2.74333403e+02, 1.09805143e+03, 1.01522923e+02]
bounds = [(0.15, 0.2), (0.15, 0.2), (1000, 5000), (100, 500), (700, 2000), (50, 200)]
x = norm_x0(x0, bounds)

# Suppose x0 is your initial 1-D ndarray of length n
n = len(x)

# Build a simplex that’s 20% off in every direction
spread = 0.2
simp = np.zeros((n+1, n))
simp[0] = x
for i in range(n):
    v = x.copy()
    v[i] += spread * max(1.0, abs(x[i]))
    simp[i+1] = v

res = minimize(
    write_eos_includes, x,
    method="Nelder-Mead",
    args = (sample_dic, experiment_dir, deck_name),
    bounds = [(0, 1) for _ in range(n)],
    options={
      "initial_simplex": simp,
      "xatol": 1e-4,     # keep these small enough to finish
      "fatol": 1e-4,
      "maxiter": 2000,
    }
)

[1.77497520e-01 1.70930780e-01 2.69506532e+03 2.74333403e+02
 1.09805143e+03 1.01522923e+02] → RMS 1.231%
[1.87497520e-01 1.70930780e-01 2.69506532e+03 2.74333403e+02
 1.09805143e+03 1.01522923e+02] → RMS 1.837%
[1.77497520e-01 1.80930780e-01 2.69506532e+03 2.74333403e+02
 1.09805143e+03 1.01522923e+02] → RMS 1.266%
[1.77497520e-01 1.70930780e-01 3.49506532e+03 2.74333403e+02
 1.09805143e+03 1.01522923e+02] → RMS 3.134%
[1.77497520e-01 1.70930780e-01 2.69506532e+03 3.54333403e+02
 1.09805143e+03 1.01522923e+02] → RMS 2.644%
[1.77497520e-01 1.70930780e-01 2.69506532e+03 2.74333403e+02
 1.35805143e+03 1.01522923e+02] → RMS 1.748%
[1.77497520e-01 1.70930780e-01 2.69506532e+03 2.74333403e+02
 1.09805143e+03 1.31522923e+02] → RMS 1.614%
[1.80830853e-01 1.74264113e-01 1.89506532e+03 3.01000070e+02
 1.18471810e+03 1.11522923e+02] → RMS 4.703%
[1.78330853e-01 1.71764113e-01 3.09506532e+03 2.81000070e+02
 1.11971810e+03 1.04022923e+02] → RMS 1.690%
[1.81108631e-01 1.74541891e-01 2.82839865e+03 